In [74]:
import json
import os
import mysql.connector
import datetime
from datetime import timedelta
import time
from pathlib import Path
from datetime import datetime
from dotenv import load_dotenv


# Start of time calculation
start_time = time.monotonic()

# Load environment variables from .env file
load_dotenv(dotenv_path='credentials.env')


# Assign environment variables
football_api_key = os.getenv('footballapikey')
mysql_host = os.getenv('mysqlhost')
mysql_user = os.getenv('mysqluser')
mysql_password = os.getenv('mysqlpassword')
mysql_db = os.getenv('mysqldb')

db = mysql.connector.connect(
        host=mysql_host,
        user=mysql_user,
        password=mysql_password,
        database=mysql_db,
        port=3306,
    )
# Drop the table if it already exists
sql_drop_fixtures = "DROP TABLE IF EXISTS `bulky`.`fixtures`;"

# And then create the table
sql_create_fixtures = '''CREATE TABLE `bulky`.`fixtures` (
  `fixture_id` INT NOT NULL,
  `fixture_utc_date` DATETIME NULL,
  `fixture_date` VARCHAR(45) NULL,
  `fixture_time` VARCHAR(45) NULL,
  `fixture_status` VARCHAR(45) NULL,
  `fixture_league_id` INT NULL,
  `fixture_round` VARCHAR(45) NULL,
  `fixture_team_home_id` INT NULL,
  `fixture_team_away_id` INT NULL,
  `fixture_team_home_name` VARCHAR(45) NULL,
  `fixture_team_away_name` VARCHAR(45) NULL,
  `fixture_goals_home` INT NULL,
  `fixture_goals_away` INT NULL,
  `fixture_team_home_is_winner` VARCHAR(45) NULL,
  `fixture_team_away_is_winner` VARCHAR(45) NULL);
    '''

In [75]:
mycursor = db.cursor()

mycursor.execute(sql_drop_fixtures)
mycursor.execute(sql_create_fixtures)
db.commit()
mycursor.close()

True

In [76]:
# Load fixtures from a json file - 41 is league one
example_json_filepath = "41.json"

# An output folder and output file name that are not used
output_folder = "output/"
new_file_name = "output-file.json"
output_filepath = output_folder + new_file_name
tuple_test = ()
Path(output_folder).mkdir(parents=True, exist_ok=True)

values_list = list()

# Load the fixtures
with open(example_json_filepath, 'r') as apiJson:
    read_api = json.load(apiJson)

    the_fixtures = read_api['FIXTURES']
    last_fixture = the_fixtures[-1]

    # The fixtures is a list of dicts
    for index, value in enumerate(the_fixtures):
        # Remove _id because that's for using a nosql database
        value.pop('_id')

        # Create a tuple of the values for the current fixture
        temp_tuple = tuple(list(value.values()))

        # Add the tuple to a list that will be used for inserting the whole list into the database
        values_list.append(temp_tuple)


In [77]:
# The 15 keys of fixtures
# 0 fixture_id
# 1 fixture_utc_date
# 2 fixture_date
# 3 fixture_time
# 4 fixture_status
# 5 fixture_league_id
# 6 fixture_round
# 7 fixture_team_home_id
# 8 fixture_team_away_id
# 9 fixture_team_home_name
# 10 fixture_team_away_name
# 11 fixture_goals_home
# 12 fixture_goals_away
# 13 fixture_team_home_is_winner
# 14 fixture_team_away_is_winner

# These were used when I was only inserting one fixture into the database. Not needed now but keeping it here anyway
# f_id = values_list[0]
# f_utc = values_list[1]
# f_date = values_list[2]
# f_time = values_list[3]
# f_status = values_list[4]
# f_league_id = values_list[5]
# f_round = values_list[6]
# f_team_home_id = values_list[7]
# f_team_away_id = values_list[8]
# f_team_home_name = values_list[9]
# f_team_away_name = values_list[10]
# f_goals_home = values_list[11]
# f_goals_away = values_list[12]
# f_team_home_is_winner = values_list[13]
# f_team_away_is_winner = values_list[14]

# A list of values to insert
# insert_list = [
#     f_id,
#     f_utc,
#     f_date,
#     f_time,
#     f_status,
#     f_league_id,
#     f_round,
#     f_team_home_id,
#     f_team_away_id,
#     f_team_home_name,
#     f_team_away_name,
#     f_goals_home,
#     f_goals_away,
#     f_team_home_is_winner,
#     f_team_away_is_winner
# ]


# A tuple of values to insert. I wrote a tuple too because I was looking at the wrpng table in mysql workbench
# and it was blank so I thought it was a problem with my code ha.
# insert_values = (f_id,
#                  f_utc,
#                  f_date,
#                  f_time,
#                  f_status,
#                  f_league_id,
#                  f_round,
#                  f_team_home_id,
#                  f_team_away_id,
#                  f_team_home_name,
#                  f_team_away_name,
#                  f_goals_home,
#                  f_goals_away,
#                  f_team_home_is_winner,
#                  f_team_away_is_winner)


sql_insert = '''insert into fixtures(
             fixture_id,
             fixture_utc_date,
             fixture_date,
             fixture_time,
             fixture_status,
             fixture_league_id,
             fixture_round,
             fixture_team_home_id,
             fixture_team_away_id,
             fixture_team_home_name,
             fixture_team_away_name,
             fixture_goals_home,
             fixture_goals_away,
             fixture_team_home_is_winner,
             fixture_team_away_is_winner)
             VALUES (%s, %s, %s, %s, %s, %s, %s,
             %s, %s, %s, %s, %s, %s, %s, %s )'''

try:
    mycursor = db.cursor()
    mycursor.executemany(sql_insert, values_list)
    mycursor.close()
    db.commit()
    db.close()
    end_time = time.monotonic()
    print(datetime.now())
    print(timedelta(seconds=end_time - start_time))
except:
    print('Sorry the database opertation did not work')

2022-03-06 22:15:09.242973
0:00:00.140000
